<a href="https://colab.research.google.com/github/martin-quinlan/gong-interview-code/blob/main/Gong_Technical_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gong Technical Support Code Examples

This notebook consolidates and adapts SQL and Python code examples for technical support automation, troubleshooting, and analysis. The original examples were separate scripts; here, they are presented in an interactive format suitable for Jupyter Notebook or Google Colab.

**Key Goals of this Notebook:**
* Provide clear explanations alongside the code.
* Make the examples runnable directly within the notebook environment.
* Use SQLite for SQL examples to ensure portability and ease of use without requiring an external database server setup.
* Demonstrate common technical support challenges and systematic approaches to solving them.

**Contents:**
1.  **Setup:** Importing necessary libraries.
2.  **Python Examples:**
    * Log Analyser for Error Pattern Detection
    * API Response Analyser for Integration Troubleshooting
3.  **SQL Examples (Adapted for SQLite):**
    * Database Schema Setup for SQLite
    * API Performance Analysis
    * Declining Usage Detection
    * Integration Sync Failures
4.  **Conclusion**

Let's begin with the setup.

In [32]:
# CELL 1: Setup - Importing necessary libraries

# This cell imports all the necessary Python libraries that will be used throughout the notebook.
# - pandas: For data manipulation and analysis, especially working with DataFrames.
# - matplotlib.pyplot: For static plotting (though we'll replace its use in the log analyser with Plotly).
# - collections.Counter: For counting hashable objects (e.g., frequency of error messages).
# - re: For regular expression operations, used in parsing and normalising text.
# - datetime, timedelta: For working with dates and times, crucial for time-window filtering.
# - os: For interacting with the operating system, like checking file existence and removing files.
# - sqlite3: For working with SQLite databases, used for the SQL examples and API log storage.
# - IPython.display.display: For a richer display of DataFrames in Jupyter environments.
# - plotly.graph_objects and plotly.subplots: For creating interactive visualisations.

import pandas as pd
import matplotlib.pyplot as plt # Kept for potential other uses, though log analyser will use Plotly
from collections import Counter
import re
from datetime import datetime, timedelta
import os
import sqlite3
from IPython.display import display # For better DataFrame display

# Plotly imports for interactive visualisations
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white" # Sets a default clean template for plotly charts

## Python Example 1: Log Analyser for Error Pattern Detection

This section focuses on analysing log files to identify error patterns. The script filters logs by a specified time window, normalises error messages to group similar ones, analyses error distribution by hour, and generates a comprehensive report along with an interactive visualisation using Plotly.

**Core Functionality:**
* **Time-based Filtering:** Focuses analysis on recent log entries.
* **Error Normalisation:** Groups similar error messages by replacing variable parts (like UUIDs, numbers, paths) with placeholders.
* **Frequency Analysis:** Determines the most common error patterns.
* **Temporal Analysis:** Identifies error bursts and hourly distribution of errors.
* **Interactive Visualisation:** Uses Plotly to create interactive charts for error distribution and top error types.

The following cell (CELL 2) will prepare a sample log file. This simulates a real-world scenario where logs are read from a file. The timestamps in the log data are dynamically generated to be recent, ensuring that the time-window filtering feature of the analyser is always demonstrable.


In [33]:
# CELL 2: Create Sample Log File for Log Analyser

# This cell dynamically creates a sample log file named 'sample_app_notebook.log'.
# The purpose is to have a realistic data source for the log analyser function.
# Timestamps in the log entries are generated based on the current date, ensuring that
# the 'time_window' feature of the analyser (which filters recent logs) can be effectively demonstrated.

# Get current year, month, and day to make logs seem recent for "yesterday" and "today"
current_dt_log = datetime.now()
y = current_dt_log.year
m_today = current_dt_log.month # month for today
d_today = current_dt_log.day # day for today

# Calculate yesterday's date components carefully to handle month/year rollovers
yesterday_dt = current_dt_log - timedelta(days=1)
y_yesterday = yesterday_dt.year
m_yesterday = yesterday_dt.month
d_yesterday = yesterday_dt.day

# Define the content of the sample log file using an f-string to insert dynamic dates.
# This log includes various levels (INFO, DEBUG, WARNING, ERROR, CRITICAL) and different error message structures.
sample_log_content = f"""
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:00:00,123] [INFO] Application started successfully by user_app_starter.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:05:00,456] [DEBUG] User 'admin_user_8573' logged in from 192.168.1.100
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:10:15,789] [WARNING] Low disk space on /var/log. Currently at 5%.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:12:00,000] [ERROR] Failed to process request ID acb123-def456-ghi789-jkl012: Database connection timeout for query on users_table. User email test@example.com.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:12:05,010] [ERROR] Another error for request ID acb123-def456-ghi789-jkl012: NullPointerException at com.example.UserService:52 during user profile update.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:13:00,200] [CRITICAL] System component 'PaymentGateway' failed to initialise. Details: Connection refused to payment.service.internal:8080 for transaction id txn_765fea.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 10:15:00,300] [INFO] Processed batch job ID job_0987_update_cache.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 23:50:00,300] [ERROR] Failed to process request ID fed321-cba654-ihg987-lkj109: Database connection timeout.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 23:52:00,300] [ERROR] User session timed out for user_id: user-xyz-789. IP: 10.0.0.52
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 23:53:00,300] [ERROR] Invalid input parameter 'user_email_address' for API endpoint /api/users/create. Expected valid email, got 'notanemail'.
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 23:54:00,300] [ERROR] File not found at path /opt/app/config/important_settings.xml. Service: ConfigLoader
[{y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} 23:55:00,300] [ERROR] Service unavailable: com.example.InventoryService failed. Caused by: java.net.ConnectException. Attempt 3 of 3.
[{y}-{m_today:02d}-{d_today:02d} 08:30:00,000] [INFO] Daily report generated successfully. report_id: report_daily_{y}{m_today:02d}{d_today:02d}.
[{y}-{m_today:02d}-{d_today:02d} 09:00:00,000] [ERROR] Request failed for customer cust_554433, order_id ord_112233: upstream service /api/v2/inventory returned 503.
[{y}-{m_today:02d}-{d_today:02d} 09:01:00,000] [ERROR] Request failed for customer cust_554433, order_id ord_112233: upstream service /api/v2/inventory returned 503.
[{y}-{m_today:02d}-{d_today:02d} 09:01:30,000] [ERROR] Request failed for customer cust_554433, order_id ord_112233: upstream service /api/v2/inventory returned 503.
[{y}-{m_today:02d}-{d_today:02d} 09:01:45,000] [ERROR] Request failed for customer cust_554433, order_id ord_112233: upstream service /api/v2/inventory returned 503.
[{y}-{m_today:02d}-{d_today:02d} 09:02:00,000] [ERROR] Request failed for customer cust_554433, order_id ord_112233: upstream service /api/v2/inventory returned 503.
"""

# Define the path for the dummy log file.
dummy_log_file_path = "sample_app_notebook.log"
# Write the sample log content to the file.
with open(dummy_log_file_path, "w") as f:
    f.write(sample_log_content.strip())

print(f"'{dummy_log_file_path}' created with sample log data for dates {y_yesterday}-{m_yesterday:02d}-{d_yesterday:02d} and {y}-{m_today:02d}-{d_today:02d}.")


'sample_app_notebook.log' created with sample log data for dates 2025-05-14 and 2025-05-15.


The next cell, CELL 3, contains the core logic for the Log Analyser. This includes:
1.  `analyse_log_file(log_file_path, time_window)`:
    * Reads the specified log file line by line.
    * Parses each line to extract timestamp, log level, and error message (if applicable).
    * Filters entries based on the `time_window` (in hours from the current time).
    * Normalises error messages using regular expressions to group similar errors.
    * Counts the frequency of each normalised error pattern.
    * Determines the distribution of errors by the hour of the day.
    * Identifies "error bursts" – sequences of multiple errors occurring in a short period.
    * Compiles a comprehensive report dictionary containing all these statistics.
    * Generates actionable recommendations based on the findings.
2.  `plot_error_distribution(report, output_path=None)`:
    * Takes the report from `analyse_log_file` as input.
    * Uses Plotly to generate two interactive subplots:
        * A bar chart showing the number of errors per hour.
        * A pie chart showing the distribution of the top 5 most frequent error patterns.
    * Optionally saves the plot to a file and displays it in the notebook.

Finally, the cell executes the `analyse_log_file` function with the dummy log file and a 48-hour time window, prints the textual report, and then calls `plot_error_distribution` to visualise the results. The temporary log file is cleaned up afterwards.

In [34]:
# CELL 3: Log Analyser Functions and Execution

def analyse_log_file(log_file_path, time_window=24):
    """
    Analyse a log file to identify error patterns within a specific time window.
    (Adapted from the original log_analyser.py script)

    Args:
        log_file_path (str): Path to the log file.
        time_window (int): Hours to look back from now for log analysis.

    Returns:
        dict: Analysis report containing error statistics and patterns,
              or an error/warning message.
    """
    # Check if the log file exists
    if not os.path.exists(log_file_path):
        return {"error": f"Log file not found: {log_file_path}"}

    # Define regex patterns for parsing log lines
    timestamp_pattern = r'\[(.*?)\]'  # Extracts content within the first square brackets (timestamp)
    level_pattern = r'\[(INFO|WARNING|ERROR|CRITICAL)\]'  # Extracts log level (INFO, WARNING, etc.)
    # Extracts error message after 'ERROR: ' or 'CRITICAL: '
    # This pattern looks for "ERROR" or "CRITICAL", then any characters (non-greedy), then ": ",
    # then captures everything until the end of the line or a newline character.
    error_pattern = r'(?:ERROR|CRITICAL).*?:\s(.*?)(?:\n|$)'

    log_entries = []  # List to store parsed log entry dictionaries
    # Calculate the time threshold for filtering logs (logs older than this will be ignored)
    time_threshold = datetime.now() - timedelta(hours=time_window)

    print(f"Analysing log file: {log_file_path}")
    print(f"Looking at entries from the past {time_window} hours (since {time_threshold.strftime('%Y-%m-%d %H:%M:%S')})")

    try:
        # Open and read the log file line by line
        with open(log_file_path, 'r') as file:
            for line_number, line in enumerate(file, 1):
                timestamp_match = re.search(timestamp_pattern, line)
                if timestamp_match:
                    timestamp_str = timestamp_match.group(1) # Extracted timestamp string
                    timestamp = None
                    # Try parsing timestamp with various common formats
                    timestamp_formats = ['%Y-%m-%d %H:%M:%S,%f', '%Y-%m-%d %H:%M:%S.%f',
                                         '%Y-%m-%d %H:%M:%S', '%d/%b/%Y:%H:%M:%S']
                    for fmt in timestamp_formats:
                        try:
                            timestamp = datetime.strptime(timestamp_str, fmt)
                            break # Parsed successfully, exit loop
                        except ValueError:
                            continue # Try next format

                    if timestamp is None: # If no format matched
                        # print(f"Warning: Could not parse timestamp on line {line_number}: {timestamp_str}")
                        continue # Skip this line if timestamp parsing fails

                    # Filter logs older than the calculated time window
                    if timestamp < time_threshold:
                        continue # Skip old log entries

                    # Extract log level
                    level_match = re.search(level_pattern, line)
                    level = level_match.group(1) if level_match else 'UNKNOWN' # Default to UNKNOWN if not found

                    # Extract error message for ERROR or CRITICAL logs
                    error_msg_extracted = None
                    if level == 'ERROR' or level == 'CRITICAL':
                        error_match = re.search(error_pattern, line)
                        if error_match:
                            error_msg_extracted = error_match.group(1).strip() # Get the captured group
                        elif 'ERROR' in line or 'CRITICAL' in line: # Fallback if main regex fails but level is identified
                            parts = line.split(f"[{level}]", 1) # Split by level marker
                            if len(parts) > 1: error_msg_extracted = parts[1].strip() # Take text after level

                    # Append parsed log entry to the list
                    log_entries.append({
                        'timestamp': timestamp, 'level': level,
                        'error_msg': error_msg_extracted, 'raw_log': line.strip(),
                        'line_number': line_number
                    })
    except Exception as e:
        return {"error": f"Failed to process log file: {str(e)}"} # Return error if file processing fails

    # If no log entries were found within the time window
    if not log_entries:
        return {"warning": "No log entries found within the specified time window.",
                "total_logs_scanned_in_window": 0, "error_logs_in_window": 0}

    # Convert list of log entries to a Pandas DataFrame for easier analysis
    log_df = pd.DataFrame(log_entries)
    total_logs_in_window = len(log_df) # Total logs processed within the window

    # Filter for ERROR and CRITICAL logs to perform error-specific analysis
    error_df_filtered = log_df[(log_df['level'] == 'ERROR') | (log_df['level'] == 'CRITICAL')].copy() # Use .copy() to avoid SettingWithCopyWarning
    error_count_in_window = len(error_df_filtered) # Total error/critical logs

    # Normalise error messages to group similar errors
    error_types = Counter() # Use Counter to store frequencies of normalised errors
    normalised_to_original = {} # Dictionary to store an example of an original error message for each normalised pattern

    if not error_df_filtered.empty and 'error_msg' in error_df_filtered.columns:
        for error in error_df_filtered['error_msg'].dropna(): # Iterate over non-null error messages
            normalised_error = str(error) # Ensure it's a string
            # Apply a series of regex substitutions to replace variable parts with placeholders
            normalised_error = re.sub(r'[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}', '<UUID>', normalised_error)
            normalised_error = re.sub(r'\b[0-9a-fA-F-]{32,}\b', '<ID_OR_HASH>', normalised_error) # General long IDs/hashes (e.g., transaction IDs)
            normalised_error = re.sub(r'\b\d+\b', '<NUM>', normalised_error) # Replace all standalone numbers
            normalised_error = re.sub(r'\/[^\s\'\"\,\;\<\>\(\)]+', '<PATH_OR_URL>', normalised_error) # Basic paths/URLs (avoids complex edge cases)
            normalised_error = re.sub(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b', '<IP_ADDRESS>', normalised_error) # IPv4 Addresses
            normalised_error = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '<EMAIL>', normalised_error) # Email addresses

            error_types[normalised_error] += 1 # Increment count for this normalised error pattern
            if normalised_error not in normalised_to_original:
                 normalised_to_original[normalised_error] = error # Store the first original error message encountered for this pattern

    # Analyse error distribution by hour
    hourly_errors = pd.Series(dtype='int64') # Initialize as an empty Series of type int64
    if not error_df_filtered.empty:
        error_df_filtered.loc[:, 'hour'] = error_df_filtered['timestamp'].dt.hour # Extract hour from timestamp
        hourly_errors = error_df_filtered.groupby('hour').size() # Count errors per hour

    # Overall log level distribution
    level_counts = log_df['level'].value_counts().to_dict() # Count occurrences of each log level

    # Identify error bursts (e.g., 5+ errors within 5 minutes)
    error_bursts = [] # List to store detected error bursts
    if not error_df_filtered.empty:
        # Sort error logs by timestamp to process them chronologically for burst detection
        sorted_error_df = error_df_filtered.sort_values('timestamp').reset_index(drop=True)
        burst_threshold_count = 5 # Minimum number of errors to constitute a burst
        burst_time_window_minutes = 5 # Time window (in minutes) for these errors to occur

        i = 0
        while i < len(sorted_error_df): # Iterate through sorted error logs
            current_burst_indices = [i] # Start a potential burst with the current error
            # Look ahead for more errors within the time window from the first error of this potential burst
            for j in range(i + 1, len(sorted_error_df)):
                # Calculate time difference between current error and the first error in the potential burst
                time_diff_seconds = (sorted_error_df.loc[j, 'timestamp'] - sorted_error_df.loc[i, 'timestamp']).total_seconds()
                if time_diff_seconds / 60 <= burst_time_window_minutes:
                    current_burst_indices.append(j) # Add to current burst if within time window
                else:
                    break # This error is outside the current burst's time window, stop looking ahead for this burst

            # If the number of errors in the potential burst meets the threshold
            if len(current_burst_indices) >= burst_threshold_count:
                start_idx, end_idx = current_burst_indices[0], current_burst_indices[-1]
                start_time = sorted_error_df.loc[start_idx, 'timestamp']
                end_time = sorted_error_df.loc[end_idx, 'timestamp']
                # Record the burst details
                error_bursts.append({
                    'start_time': start_time, 'end_time': end_time,
                    'duration_minutes': (end_time - start_time).total_seconds() / 60,
                    'error_count': len(current_burst_indices),
                    'sample_errors': sorted_error_df.loc[current_burst_indices[:3], 'error_msg'].tolist() # Sample first 3 errors
                })
                i = current_burst_indices[-1] + 1 # Move index past this detected burst
            else:
                i += 1 # Move to the next log entry to start checking for a new potential burst

    # Prepare the final report dictionary
    report = {
        'total_logs_scanned_in_window': total_logs_in_window,
        'error_logs_in_window': error_count_in_window,
        'error_percentage': (error_count_in_window / total_logs_in_window * 100) if total_logs_in_window > 0 else 0,
        'top_error_types': [{ # List of top 10 error patterns with details
            'pattern': err, 'count': cnt,
            'percentage': (cnt / error_count_in_window * 100) if error_count_in_window > 0 else 0,
            'example': normalised_to_original.get(err, 'N/A') # Original example for the pattern
        } for err, cnt in error_types.most_common(10)],
        'hourly_error_distribution': hourly_errors.reindex(range(24), fill_value=0).to_dict(), # Ensure all hours 0-23 are present
        'level_distribution': level_counts,
        'error_bursts': error_bursts
    }

    # Generate actionable recommendations based on findings
    recommendations = []
    if error_count_in_window > 0: # Only generate recommendations if errors exist
        # Recommendation for frequent error patterns
        for error_detail in report['top_error_types'][:3]: # Focus on top 3 patterns
            if error_detail['count'] > error_count_in_window * 0.1: # If an error type is >10% of total errors
                recommendations.append(f"Investigate frequent error pattern ({error_detail['percentage']:.1f}% of errors): {error_detail['pattern'][:100]}...")
    # Recommendation for error bursts
    if error_bursts:
        recommendations.append(f"Examine {len(error_bursts)} error burst(s). This might indicate cascading failures or system under high load.")
        for i_burst, burst in enumerate(error_bursts[:2], 1): # Provide details for the first 2 bursts
            recommendations.append(f"  Burst {i_burst}: {burst['error_count']} errors in {burst['duration_minutes']:.1f} mins around {burst['start_time'].strftime('%H:%M')}")
    # Recommendation for hourly error concentration
    if not hourly_errors.empty:
        peak_hour = hourly_errors.idxmax() # Hour with the most errors
        peak_count = hourly_errors.max()
        # If peak hour has >30% of all errors and there are a reasonable number of errors (e.g., >2)
        if error_count_in_window > 0 and peak_count > error_count_in_window * 0.3 and peak_count > 2 :
            recommendations.append(f"High error concentration at hour {peak_hour:02d}:00 ({peak_count} errors). Check for scheduled jobs or peak load.")

    report['recommendations'] = recommendations if recommendations else ["No specific urgent recommendations based on current patterns within the time window."]
    return report

def plot_error_distribution(report, output_path=None):
    """
    Generate interactive visualisations of error patterns using Plotly.

    Args:
        report (dict): Analysis report from analyse_log_file.
        output_path (str, optional): Path to save the visualisation as an HTML file.
    """
    # Check if there's data to visualize
    if 'error' in report or report.get('total_logs_scanned_in_window', 0) == 0 or report.get('error_logs_in_window', 0) == 0:
        print("No data or no errors to visualise for the log analysis.")
        return

    # Create subplots: 1 row, 2 columns. Second plot (pie) will span more relative width.
    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.4, 0.6], # Adjust relative widths
        specs=[[{'type':'xy'}, {'type':'domain'}]], # Specify plot types: xy for bar, domain for pie
        subplot_titles=("Error Distribution by Hour", "Top 5 Error Types")
    )

    # --- Plot 1: Error distribution by hour (Bar Chart) ---
    hourly_dist = report['hourly_error_distribution']
    hours = list(hourly_dist.keys())
    counts = list(hourly_dist.values())

    fig.add_trace(
        go.Bar(x=hours, y=counts, name='Errors per Hour', marker_color='rgb(26, 118, 255)'), # marker_color is a parameter name
        row=1, col=1
    )
    fig.update_xaxes(title_text="Hour of Day (0-23)", tickmode='linear', dtick=1, row=1, col=1) # Ensure all hour ticks
    fig.update_yaxes(title_text="Number of Errors", row=1, col=1)

    # --- Plot 2: Top error types (Pie Chart) ---
    top_errors = report['top_error_types'][:5] # Get top 5 error patterns
    # Filter out any patterns that might have zero count if they somehow make it here
    top_errors_filtered = [e for e in top_errors if e['count'] > 0]

    if top_errors_filtered:
        # Prepare labels (shortened patterns for readability) and sizes for the pie chart
        labels = [f"{e['pattern'][:45]}... ({e['count']})" for e in top_errors_filtered]
        sizes = [e['count'] for e in top_errors_filtered]

        fig.add_trace(
            go.Pie(labels=labels, values=sizes, name='Top Errors', hole=.3), # hole makes it a donut chart
            row=1, col=2
        )
        fig.update_traces(textinfo='percent+label', textfont_size=10, row=1, col=2)
    else:
        # If no significant error types, display a message on the plot
        fig.add_annotation(text="No significant error types to display.",
                           xref="paper", yref="paper",
                           x=0.75, y=0.5, showarrow=False, # x=0.75 positions it in the centre of the second subplot area
                           font=dict(size=12), row=1, col=2)

    # Update layout for the entire figure
    fig.update_layout(
        title_text='Interactive Log Analysis Visualisations',
        height=500, # Adjust height
        showlegend=False, # Legend is part of pie chart labels or can be added separately if needed
        margin=dict(l=50, r=50, t=80, b=50) # Adjust margins
    )

    # Save to HTML if output_path is provided
    if output_path:
        # Ensure output path ends with .html for plotly interactive file
        if not output_path.endswith(".html"):
            output_path += ".html"
        pio.write_html(fig, file=output_path, auto_open=False)
        print(f"Log analysis interactive visualisation saved to {output_path}")

    fig.show() # Display interactive plot in notebook

# --- Run the Log Analyser ---
# Analyse logs from the last 48 hours from the current time.
# This time window ensures that the dynamically generated logs for "yesterday" and "today" are included.
log_analysis_report = analyse_log_file(dummy_log_file_path, time_window=48)

# Check for errors or warnings from the analysis function
if 'error' in log_analysis_report:
    print(f"Error during log analysis: {log_analysis_report['error']}")
elif 'warning' in log_analysis_report:
    print(f"Warning during log analysis: {log_analysis_report['warning']}")
else:
    # Print the textual part of the report
    print("\n--- Log Analysis Report ---")
    print(f"Total logs scanned in window: {log_analysis_report['total_logs_scanned_in_window']}")
    print(f"Error logs found in window: {log_analysis_report['error_logs_in_window']} ({log_analysis_report['error_percentage']:.2f}%)")

    print("\nTop Error Patterns (up to 5):")
    if log_analysis_report['top_error_types']:
        for i, error in enumerate(log_analysis_report['top_error_types'][:5], 1): # Display top 5
            print(f"  {i}. Count: {error['count']} ({error['percentage']:.1f}%) - Pattern: {error['pattern'][:100]}...")
            print(f"     Example: {error['example'][:100]}...")
    else:
        print("  No specific error patterns found.")

    print("\nError Bursts:")
    if log_analysis_report['error_bursts']:
        for i, burst in enumerate(log_analysis_report['error_bursts'],1):
            print(f"  Burst {i}: {burst['error_count']} errors from {burst['start_time']} to {burst['end_time']} ({burst['duration_minutes']:.2f} min)")
            if burst['sample_errors']:
                 print(f"     Samples: {burst['sample_errors'][0][:100]}...") # Show first sample error of the burst
    else:
        print("  No significant error bursts detected.")

    print("\nRecommendations:")
    if log_analysis_report['recommendations']:
        for i, rec in enumerate(log_analysis_report['recommendations'], 1):
            print(f"  {i}. {rec}")
    else:
        print("  No recommendations at this time.")

    print("\n--- End of Log Analysis Report ---")

    # Generate and show the interactive Plotly plot
    plot_error_distribution(log_analysis_report, output_path="interactive_log_error_analysis.html")

# Clean up the dummy log file created earlier
if os.path.exists(dummy_log_file_path):
    os.remove(dummy_log_file_path)
    print(f"\nTemporary log file '{dummy_log_file_path}' removed.")

Analysing log file: sample_app_notebook.log
Looking at entries from the past 48 hours (since 2025-05-13 10:08:32)

--- Log Analysis Report ---
Total logs scanned in window: 18
Error logs found in window: 13 (72.22%)

Top Error Patterns (up to 5):
  1. Count: 5 (38.5%) - Pattern: upstream service <PATH_OR_URL> returned <NUM>....
     Example: upstream service /api/v2/inventory returned 503....
  2. Count: 1 (7.7%) - Pattern: Database connection timeout for query on users_table. User email <EMAIL>....
     Example: Database connection timeout for query on users_table. User email test@example.com....
  3. Count: 1 (7.7%) - Pattern: NullPointerException at com.example.UserService:<NUM> during user profile update....
     Example: NullPointerException at com.example.UserService:52 during user profile update....
  4. Count: 1 (7.7%) - Pattern: Connection refused to payment.service.internal:<NUM> for transaction id txn_765fea....
     Example: Connection refused to payment.service.internal:80


Temporary log file 'sample_app_notebook.log' removed.


## Python Example 2: API Response Analyser

This section provides tools to analyse API response logs. The goal is to identify patterns in failed requests, pinpoint slow endpoints, and understand time-based trends in API performance. This is crucial for troubleshooting integration issues.

**Methodology:**
1.  **Data Source:** Mock API logs are stored in a temporary SQLite database.
2.  **Data Fetching:** Logs are fetched for a specific customer and date range.
3.  **Analysis:** The script calculates:
    * Overall metrics (total requests, success rate, average response time, error count).
    * Summary of response codes (count, average/median/max response times per status code).
    * Performance of API endpoints (count, average/median/max response times, success rate per endpoint).
    * Most common error patterns (grouping by endpoint, status code, and error message).
    * Hourly distribution of errors.
    * Correlation between response time buckets and error rates.
4.  **Reporting:** Results are presented in Pandas DataFrames and a summary dictionary, along with actionable recommendations.

The next cell (CELL 4) sets up the SQLite database (`api_logs_notebook.db`) and populates it with sample API log data. This database is created in the notebook's local environment and will be removed after the analysis to ensure a clean state for subsequent runs.
```python

In [35]:
# CELL 4: API Response Analyser - Database Setup

# Define the name for the SQLite database file.
DB_API_LOGS_NAME = 'api_logs_notebook.db'

def setup_api_logs_db_notebook():
    """
    Creates and populates an SQLite database with mock API log data.
    The database is removed if it already exists to ensure a fresh start for each notebook run.
    """
    # Remove the database file if it already exists to start fresh.
    if os.path.exists(DB_API_LOGS_NAME):
        os.remove(DB_API_LOGS_NAME)

    # Connect to (or create) the SQLite database.
    conn = sqlite3.connect(DB_API_LOGS_NAME)
    cursor = conn.cursor()

    # Define the table schema for API logs.
    # Timestamps are stored as TEXT in ISO8601 format for SQLite compatibility and readability.
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS api_logs (
        request_id TEXT PRIMARY KEY,
        timestamp TEXT NOT NULL,
        endpoint TEXT NOT NULL,
        status_code INTEGER NOT NULL,
        response_time_ms INTEGER NOT NULL,
        error_message TEXT,
        customer_id TEXT NOT NULL
    )
    ''')

    logs_data = [] # List to hold generated log entries
    # Use a fixed start date for consistent and predictable sample data generation.
    analysis_start_date = datetime.strptime('2024-01-01', '%Y-%m-%d')

    # Generate 250 mock API log entries.
    for i in range(250):
        customer = 'CUST78901' # Sample customer ID for all logs in this example.
        is_error = i % 6 == 0 # Roughly 1 in 6 requests will be an error, for variety.
        status_code_val = 200 # Default to HTTP 200 OK (success).
        error_msg_val = None # Default to no error message.

        if is_error:
            error_type_val = i % 5 # Vary error types to simulate different failure modes.
            if error_type_val == 0: status_code_val = 400; error_msg_val = 'Bad Request: Invalid parameter X.'
            elif error_type_val == 1: status_code_val = 401; error_msg_val = 'Unauthorised: Missing API Key.'
            elif error_type_val == 2: status_code_val = 404; error_msg_val = 'Not Found: Resource /items/item<NUM> does not exist.'
            elif error_type_val == 3: status_code_val = 500; error_msg_val = 'Internal Server Error: Null pointer in backend service.'
            else: status_code_val = 503; error_msg_val = 'Service Unavailable: Upstream dependency timeout.'

        # Vary endpoints to simulate different API calls.
        endpoint_val = f'/api/v1/{"resourceA" if i % 4 == 0 else "resourceB" if i % 4 == 1 else "resourceC" if i % 4 == 2 else "resourceD"}'

        # Append the generated log entry as a tuple to the list.
        logs_data.append((
            f'req-{i:04d}', # Padded request_id for consistent formatting.
            (analysis_start_date + timedelta(hours=i*1, minutes=i*10)).strftime('%Y-%m-%d %H:%M:%S'), # Staggered timestamps.
            endpoint_val,
            status_code_val,
            # Varied response times, slightly higher for error status codes. Ensure positive response time.
            max(50, 100 + (i * 7) % 600 + (i % 11 * 30) - (status_code_val // 100 * 20)),
            error_msg_val,
            customer
        ))

    # Insert all generated log entries into the database in a single batch.
    # 'INSERT OR IGNORE' will skip insertion if a request_id (PRIMARY KEY) already exists.
    cursor.executemany("INSERT OR IGNORE INTO api_logs VALUES (?, ?, ?, ?, ?, ?, ?)", logs_data)
    conn.commit() # Commit the changes to the database.
    conn.close() # Close the database connection.

    print(f"Database '{DB_API_LOGS_NAME}' created and populated with {len(logs_data)} entries.")

# Execute the database setup function.
setup_api_logs_db_notebook()

Database 'api_logs_notebook.db' created and populated with 250 entries.


CELL 5 below contains the main logic for the API Response Analyser. It defines two key functions:
1.  `fetch_api_logs_from_db_notebook(customer_id, start_date_str, end_date_str)`:
    * Connects to the SQLite database (`api_logs_notebook.db`) created in the previous step.
    * Constructs and executes an SQL query to fetch API logs for the specified `customer_id` and within the given date range (`start_date_str` to `end_date_str`).
    * The date range handling is inclusive for the start date and exclusive for the end date (i.e., up to the beginning of the day after `end_date_str`).
    * Returns the fetched logs as a Pandas DataFrame, with the `timestamp` column converted to datetime objects.
2.  `analyse_api_responses_notebook(customer_id, start_date, end_date)`:
    * Calls `fetch_api_logs_from_db_notebook` to get the relevant log data.
    * If no data is found, it returns an error message.
    * Performs various analyses on the DataFrame:
        * Calculates overall metrics (total requests, success rate, average response time, total error count).
        * Summarises response codes (count and response time statistics per status code).
        * Analyses endpoint performance (request count, response time statistics, success rate, and error count per endpoint).
        * Identifies top error patterns by grouping errors by endpoint, status code, and error message.
        * Determines the hourly distribution of errors.
        * Analyses the relationship between response time buckets and error rates.
    * Compiles all findings into a structured report dictionary, which includes DataFrames for detailed tables and dictionaries for other metrics.
    * Generates a list of actionable recommendations based on the analysis (e.g., investigating endpoints with high error rates or slow response times, checking peak error hours).

The cell then runs this analyser for a sample customer ("CUST78901") and a defined date range, printing a summary of the report and displaying key DataFrames. Finally, it cleans up by removing the temporary SQLite database file.


In [37]:
# CELL 5: API Response Analyser - Main Functions and Execution

def fetch_api_logs_from_db_notebook(customer_id, start_date_str, end_date_str):
    """
    Fetches API logs from the SQLite DB for a given customer and date range.

    Args:
        customer_id (str): The ID of the customer whose logs are to be fetched.
        start_date_str (str): The start date of the period (YYYY-MM-DD).
        end_date_str (str): The end date of the period (YYYY-MM-DD).

    Returns:
        pd.DataFrame: A DataFrame containing the fetched API logs.
    """
    # Connect to the SQLite database.
    conn = sqlite3.connect(DB_API_LOGS_NAME)

    # Prepare date strings for the SQL query.
    # The query will fetch logs from the start of start_date_str up to (but not including) the start of the day after end_date_str.
    effective_start_date_str = start_date_str + " 00:00:00" # Start of the day
    effective_end_date_dt = datetime.strptime(end_date_str, '%Y-%m-%d') + timedelta(days=1) # Day after end_date
    effective_end_date_str = effective_end_date_dt.strftime('%Y-%m-%d %H:%M:%S') # Start of the day after end_date

    # SQL query to select logs based on customer_id and timestamp range.
    query = """
        SELECT request_id, timestamp, endpoint, status_code, response_time_ms, error_message
        FROM api_logs
        WHERE customer_id = ? AND timestamp >= ? AND timestamp < ?
        ORDER BY timestamp
    """ # Logs are ordered by timestamp.

    # Execute the query and load results directly into a Pandas DataFrame.
    df = pd.read_sql_query(query, conn, params=[customer_id, effective_start_date_str, effective_end_date_str])
    conn.close() # Close the database connection.

    # Convert the 'timestamp' column from string to datetime objects if the DataFrame is not empty.
    if not df.empty:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

def analyse_api_responses_notebook(customer_id, start_date, end_date):
    """
    Analyses API logs fetched from the database and returns a structured report.

    Args:
        customer_id (str): Customer ID for the analysis.
        start_date (str): Start date for the analysis period (YYYY-MM-DD).
        end_date (str): End date for the analysis period (YYYY-MM-DD).

    Returns:
        dict: A dictionary containing various analysis results (metrics, DataFrames, recommendations).
              Returns an error dictionary if no data is found.
    """
    # Fetch the API logs for the specified customer and date range.
    df = fetch_api_logs_from_db_notebook(customer_id, start_date, end_date)

    # Handle cases where no logs are found.
    if df.empty:
        print(f"No API logs found for customer {customer_id} between {start_date} and {end_date}.")
        return {"error": "No data available for the specified criteria."}

    # --- Overall Metrics Calculation ---
    total_requests = len(df)
    successful_requests_df = df[df['status_code'] < 400] # Requests with status codes < 400 are typically successful.
    error_requests_df = df[df['status_code'] >= 400].copy() # Requests with status codes >= 400 are errors. Use .copy() for modifications.
    success_rate = (len(successful_requests_df) / total_requests * 100) if total_requests > 0 else 0
    avg_response_time = df['response_time_ms'].mean()
    error_count = len(error_requests_df)

    # --- Response Code Summary ---
    # Group by status_code to get counts and response time statistics for each code.
    response_summary_df = df.groupby('status_code').agg(
        request_count=('request_id', 'count'),
        mean_response_time_ms=('response_time_ms', 'mean'),
        median_response_time_ms=('response_time_ms', 'median'),
        max_response_time_ms=('response_time_ms', 'max')
    ).reset_index().sort_values(by='request_count', ascending=False) # Sort by request count.

    # --- Endpoint Performance Analysis ---
    # Group by endpoint to analyse performance metrics for each API endpoint.
    endpoint_performance_df = df.groupby('endpoint').agg(
        request_count=('request_id', 'count'),
        mean_response_time_ms=('response_time_ms', 'mean'),
        median_response_time_ms=('response_time_ms', 'median'),
        max_response_time_ms=('response_time_ms', 'max'),
        success_rate_percent=('status_code', lambda x: (x < 400).mean() * 100), # Calculate success rate.
        error_count_col=('status_code', lambda x: (x >=400).sum()) # Calculate error count per endpoint.
    ).reset_index().sort_values(by='mean_response_time_ms', ascending=False) # Sort by mean response time.
    endpoint_performance_df.rename(columns={'error_count_col': 'error_count'}, inplace=True) # Rename column for clarity.


    # --- Top Error Patterns Analysis ---
    # Group by endpoint, status_code, and error_message to find common error patterns.
    top_errors_df = pd.DataFrame() # Initialize as an empty DataFrame.
    if not error_requests_df.empty:
        top_errors_df = error_requests_df.groupby(['endpoint', 'status_code', 'error_message']).size().reset_index(name='count')
        top_errors_df = top_errors_df.sort_values('count', ascending=False) # Sort by frequency.

    # --- Hourly Error Distribution ---
    # Analyse how errors are distributed across different hours of the day.
    hourly_error_distribution = {} # Initialize as an empty dictionary.
    if not error_requests_df.empty:
        error_requests_df.loc[:, 'hour_of_day'] = error_requests_df['timestamp'].dt.hour # Extract hour.
        hourly_errors = error_requests_df.groupby('hour_of_day').size().reindex(range(24), fill_value=0) # Ensure all hours 0-23 are present.
        hourly_error_distribution = hourly_errors.to_dict()
    else: # If no errors, create a dictionary with 0 counts for all hours.
        hourly_error_distribution = {h: 0 for h in range(24)}


    # --- Response Time vs. Errors Analysis ---
    # Analyse if there's a correlation between response time buckets and error rates.
    time_vs_errors = {} # Initialize as an empty dictionary.
    if total_requests > 0: # Proceed only if there are requests.
        time_thresholds = [0, 100, 250, 500, 1000, 2000, float('inf')] # Define response time buckets in milliseconds.
        for i in range(len(time_thresholds)-1):
            lower, upper = time_thresholds[i], time_thresholds[i+1]
            time_range_label = f"{lower}-{upper if upper != float('inf') else 'MAX'} ms"

            # Filter requests within this response time bucket.
            bucket_df = df[(df['response_time_ms'] > lower) & (df['response_time_ms'] <= upper)]
            count_in_bucket = len(bucket_df)
            errors_in_bucket = len(bucket_df[bucket_df['status_code'] >= 400])
            error_rate_in_bucket = (errors_in_bucket / count_in_bucket * 100) if count_in_bucket > 0 else 0

            time_vs_errors[time_range_label] = {
                'total_requests_in_bucket': count_in_bucket,
                'errors_in_bucket': errors_in_bucket,
                'error_rate_percent': error_rate_in_bucket
            }

    # --- Report Assembly ---
    # Compile all analysis results into a single report dictionary.
    report = {
        'analysis_period': {'start_date': start_date, 'end_date': end_date, 'customer_id': customer_id},
        'overall_metrics': {'total_requests': total_requests, 'success_rate_percent': success_rate,
                            'average_response_time_ms': avg_response_time, 'total_error_count': error_count},
        'response_code_summary_df': response_summary_df, # DataFrame
        'endpoint_performance_df': endpoint_performance_df, # DataFrame
        'top_errors_df': top_errors_df, # DataFrame
        'hourly_error_distribution_dict': hourly_error_distribution, # Dictionary
        'response_time_vs_errors_dict': time_vs_errors # Dictionary
    }

    # --- Generate Recommendations ---
    recommendations = []
    if not endpoint_performance_df.empty:
        # Identify endpoints with high error rates (e.g., success rate < 85% and more than 10 requests).
        problem_endpoints_error = endpoint_performance_df[(endpoint_performance_df['success_rate_percent'] < 85) & (endpoint_performance_df['request_count'] > 10)]
        for _, row in problem_endpoints_error.iterrows():
            recommendations.append(f"Investigate high error rate ({100-row['success_rate_percent']:.1f}%) for endpoint: {row['endpoint']} ({row['error_count']} errors).")
        # Identify slow endpoints (e.g., average response time > 1000ms and more than 5 requests).
        slow_endpoints = endpoint_performance_df[(endpoint_performance_df['mean_response_time_ms'] > 1000) & (endpoint_performance_df['request_count'] > 5)]
        for _, row in slow_endpoints.iterrows():
            recommendations.append(f"Optimise performance for slow endpoint: {row['endpoint']} (avg: {row['mean_response_time_ms']:.0f}ms).")

    # Check for peak error hours.
    if hourly_error_distribution:
        hourly_series = pd.Series(hourly_error_distribution)
        if not hourly_series.empty and error_count > 0: # Ensure there are errors to analyse.
            peak_hour = hourly_series.idxmax() # Hour with the most errors.
            # If peak hour has a significant portion of errors (e.g., >25% of total errors and at least 5 errors).
            if hourly_series.max() > error_count * 0.25 and error_count > 5:
                 recommendations.append(f"Investigate potential issues during peak error hour: {peak_hour:02d}:00 (count: {hourly_series.max()}).")

    report['recommendations_list'] = recommendations if recommendations else ["API performance appears generally stable within the analysed parameters."]
    return report

# --- Run the API Response Analyser ---
# Define customer ID and date range for the analysis. These should align with the data generated.
api_customer_id = "CUST78901"
api_start_date = "2024-01-01"
api_end_date = "2024-01-20"   # Analyse a subset of the generated data.

# Perform the analysis.
api_analysis_report = analyse_api_responses_notebook(api_customer_id, api_start_date, api_end_date)

# Print the analysis results.
if 'error' in api_analysis_report:
    print(f"Error during API analysis: {api_analysis_report['error']}")
else:
    print("\n--- API Response Analysis Report ---")
    print(f"Customer: {api_analysis_report['analysis_period']['customer_id']}, Period: {api_analysis_report['analysis_period']['start_date']} to {api_analysis_report['analysis_period']['end_date']}")
    om = api_analysis_report['overall_metrics'] # Shorthand for overall metrics.
    print(f"Overall: Total Requests: {om['total_requests']}, Success: {om['success_rate_percent']:.2f}%, Avg Response: {om['average_response_time_ms']:.2f}ms, Errors: {om['total_error_count']}")

    print("\nResponse Code Summary:")
    display(api_analysis_report['response_code_summary_df']) # Use display for rich DataFrame output in notebooks.

    print("\nEndpoint Performance (Top 5 by Avg Response Time):")
    display(api_analysis_report['endpoint_performance_df'].head()) # Display top 5.

    print("\nTop Error Patterns (Top 5):")
    display(api_analysis_report['top_errors_df'].head()) # Display top 5.

    print("\nHourly Error Distribution (Sample - first 6 hours):")
    # Print a sample of the hourly error distribution.
    print({k: v for i, (k,v) in enumerate(api_analysis_report['hourly_error_distribution_dict'].items()) if i < 6})

    print("\nResponse Time vs. Errors (Sample - first 3 buckets):")
    # Print a sample of the response time vs. errors analysis.
    for k, v in list(api_analysis_report['response_time_vs_errors_dict'].items())[:3]:
        print(f"  {k}: Total={v['total_requests_in_bucket']}, Errors={v['errors_in_bucket']}, ErrorRate={v['error_rate_percent']:.1f}%")

    print("\nRecommendations:")
    if api_analysis_report['recommendations_list']:
        for rec in api_analysis_report['recommendations_list']: print(f"- {rec}")
    else:
        print("No specific recommendations at this time.") # Message if no recommendations were generated.
    print("--- End of API Response Analysis Report ---")

# Clean up the dummy API log DB file created earlier.
if os.path.exists(DB_API_LOGS_NAME):
    os.remove(DB_API_LOGS_NAME)
    print(f"\nTemporary API log database '{DB_API_LOGS_NAME}' removed.")



--- API Response Analysis Report ---
Customer: CUST78901, Period: 2024-01-01 to 2024-01-20
Overall: Total Requests: 250, Success: 83.20%, Avg Response: 491.38ms, Errors: 42

Response Code Summary:


,status_code,request_count,mean_response_time_ms,median_response_time_ms,max_response_time_ms
0,200,208,499.918269,506.0,908
1,400,9,416.666667,470.0,770
2,401,9,452.000000,452.0,662
3,404,8,441.500000,389.0,794
4,500,8,478.500000,501.0,666
5,503,8,460.500000,423.0,798



Endpoint Performance (Top 5 by Avg Response Time):


,endpoint,request_count,mean_response_time_ms,median_response_time_ms,max_response_time_ms,success_rate_percent,error_count
1,/api/v1/resourceB,63,505.476190,507.0,895,100.000000,0
3,/api/v1/resourceD,62,500.967742,503.0,871,100.000000,0
2,/api/v1/resourceC,62,479.774194,473.0,818,66.129032,21
0,/api/v1/resourceA,63,479.269841,470.0,908,66.666667,21



Top Error Patterns (Top 5):


,endpoint,status_code,error_message,count
0,/api/v1/resourceA,400,Bad Request: Invalid parameter X.,5
6,/api/v1/resourceC,401,Unauthorised: Missing API Key.,5
1,/api/v1/resourceA,401,Unauthorised: Missing API Key.,4
2,/api/v1/resourceA,404,Not Found: Resource /items/item<NUM> does not ...,4
4,/api/v1/resourceA,503,Service Unavailable: Upstream dependency timeout.,4



Hourly Error Distribution (Sample - first 6 hours):
{0: 2, 1: 2, 2: 2, 3: 1, 4: 2, 5: 2}

Response Time vs. Errors (Sample - first 3 buckets):
  0-100 ms: Total=4, Errors=1, ErrorRate=25.0%
  100-250 ms: Total=28, Errors=7, ErrorRate=25.0%
  250-500 ms: Total=95, Errors=17, ErrorRate=17.9%

Recommendations:
- Investigate high error rate (33.9%) for endpoint: /api/v1/resourceC (21 errors).
- Investigate high error rate (33.3%) for endpoint: /api/v1/resourceA (21 errors).
--- End of API Response Analysis Report ---

Temporary API log database 'api_logs_notebook.db' removed.


## SQL Examples (Adapted for SQLite)

This section presents SQL queries originally designed for PostgreSQL, now adapted to run against an SQLite database. This adaptation ensures the notebook is self-contained and easily runnable without needing an external database server. SQLite is a lightweight, file-based database engine, and Python has built-in support for it via the `sqlite3` module.

**Key Adaptations for SQLite:**
* **Data Types:** PostgreSQL's `SERIAL PRIMARY KEY` is typically `INTEGER PRIMARY KEY AUTOINCREMENT` in SQLite. Timestamps and dates are stored as `TEXT` in ISO8601 format (e.g., "YYYY-MM-DD HH:MM:SS") for better human readability and cross-platform compatibility, though SQLite can also store them as numbers (Julian day or Unix epoch).
* **Date/Time Functions:** PostgreSQL-specific functions like `DATEADD`, `DATEDIFF`, and `INTERVAL` arithmetic are replaced with SQLite's date and time functions such as `date()`, `datetime()`, `strftime()`, and modifiers like `'+X days'`. Functions like `CURRENT_DATE` and `CURRENT_TIMESTAMP` are generally available, though `datetime('now', 'localtime')` is often used to get the current timestamp adjusted for local time.
* **Random Data Generation:** `random()` in PostgreSQL (returning a float between 0 and 1) is different from SQLite's `random()` (returning a large signed integer). We use `abs(random())` in SQLite to get positive random numbers for data generation.
* **Aggregate Functions with CASE:** While `COUNT(CASE WHEN condition THEN 1 END)` works in SQLite (counting non-NULL results), `SUM(CASE WHEN condition THEN 1 ELSE 0 END)` is sometimes used for clarity or if a zero count is explicitly desired when the condition is never met (though `COALESCE(COUNT(...), 0)` also achieves this).

The next cell (CELL 6) will set up an **in-memory** SQLite database. This means the database and its data will only exist for the duration of the current notebook session and will be lost when the notebook kernel is shut down. It then creates the necessary table schemas and populates them with sample data relevant to the SQL queries that follow.

In [38]:
# CELL 6: SQL Examples - Database Setup (SQLite In-Memory)

# This cell sets up an SQLite database in memory.
# An in-memory database is fast and temporary, ideal for demonstrations within a notebook.
# All data will be lost when the notebook kernel stops.
# To persist the database, replace ':memory:' with a file path (e.g., 'gong_sql_examples.db').
conn_sql_notebook = sqlite3.connect(':memory:')
cursor_sql_notebook = conn_sql_notebook.cursor() # Create a cursor object to execute SQL commands.

def execute_sqlite_script(script_content, cursor, connection):
    """
    Executes a multi-statement SQL script for SQLite.

    Args:
        script_content (str): A string containing one or more SQL statements separated by semicolons.
        cursor (sqlite3.Cursor): The database cursor.
        connection (sqlite3.Connection): The database connection.
    """
    try:
        cursor.executescript(script_content) # Executes multiple SQL statements.
        connection.commit() # Commit changes to the database.
        print("SQLite script for database setup executed successfully.")
    except sqlite3.Error as e:
        print(f"SQLite error during script execution: {e}") # Print error message if execution fails.

# Combined DDL (Data Definition Language) and DML (Data Manipulation Language) script for SQLite.
# This script creates tables and inserts sample data.
# Timestamps and dates are stored as TEXT in "YYYY-MM-DD HH:MM:SS" or "YYYY-MM-DD" format.
sqlite_setup_script_notebook = """
-- Drop tables if they already exist to ensure a clean setup.
DROP TABLE IF EXISTS integration_sync_records;
DROP TABLE IF EXISTS calls;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS api_request_logs;

-- Create table for API request logs.
CREATE TABLE api_request_logs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT, -- Auto-incrementing primary key.
    timestamp TEXT NOT NULL, -- ISO8601 strings "YYYY-MM-DD HH:MM:SS".
    endpoint_path TEXT NOT NULL,
    response_time_ms INTEGER NOT NULL,
    status_code INTEGER NOT NULL,
    customer_id TEXT
);

-- Insert sample data into api_request_logs.
-- Using fixed dates here for predictability in the API Performance Analysis query.
INSERT INTO api_request_logs (timestamp, endpoint_path, response_time_ms, status_code, customer_id) VALUES
('2024-07-01 10:00:00', '/api/v1/users', 150, 200, 'CUSTSQL01'),
('2024-07-01 10:01:00', '/api/v1/users', 200, 200, 'CUSTSQL02'),
('2024-07-01 10:02:00', '/api/v1/products', 500, 200, 'CUSTSQL01'),
('2024-07-01 10:04:00', '/api/v1/products', 750, 404, 'CUSTSQL02'), -- Example of an error response.
('2024-07-01 10:05:00', '/api/v1/orders', 1200, 201, 'CUSTSQL01'),
('2024-07-03 12:00:00', '/api/v1/users', 120, 200, 'CUSTSQL02'),
('2024-07-06 15:00:00', '/api/v1/users', 3000, 503, 'CUSTSQL05'); -- Example of an error and slow response.

-- Create table for users.
CREATE TABLE users (
    user_id TEXT PRIMARY KEY,
    full_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL, -- Email should be unique.
    department TEXT
);

-- Insert sample users.
INSERT INTO users (user_id, full_name, email, department) VALUES
('SQLUSER001', 'SQL Alice', 'sql_alice@example.com', 'Sales'),
('SQLUSER002', 'SQL Bob', 'sql_bob@example.com', 'Support'),
('SQLUSER003', 'SQL Charlie', 'sql_charlie@example.com', 'Sales'),
('SQLUSER004', 'SQL Diana', 'sql_diana@example.com', 'Customer Success');

-- Create table for calls.
CREATE TABLE calls (
    call_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT REFERENCES users(user_id), -- Foreign key to users table.
    call_date TEXT NOT NULL, -- Date of the call "YYYY-MM-DD".
    started_at TEXT NOT NULL, -- Timestamp when call started "YYYY-MM-DD HH:MM:SS".
    duration INTEGER, -- Duration in minutes.
    recording_status TEXT -- e.g., 'COMPLETED', 'PROCESSING'.
);

-- Insert calls data using SQLite's date functions relative to 'now'.
-- This makes the declining usage query results dynamic and demonstrable.
-- For USER SQLUSER001 (simulating activity then a slight decline).
-- Previous period (31-60 days ago): 15 calls.
INSERT INTO calls (user_id, call_date, started_at, duration, recording_status)
SELECT 'SQLUSER001',
       date('now', '-' || (abs(random()) % 29 + 31) || ' days'), -- Random date 31-59 days ago.
       datetime('now', '-' || (abs(random()) % 29 + 31) || ' days', '+' || (abs(random()) % 10) || ' hours'), -- Random timestamp.
       abs(random())%50+10, 'COMPLETED'
FROM (SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10 UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15);

-- Current period (last 30 days): 11 calls (a decline from 15).
INSERT INTO calls (user_id, call_date, started_at, duration, recording_status)
SELECT 'SQLUSER001',
       date('now', '-' || (abs(random()) % 30) || ' days'), -- Random date in last 30 days.
       datetime('now', '-' || (abs(random()) % 30) || ' days', '+' || (abs(random()) % 10) || ' hours'),
       abs(random())%50+10, 'COMPLETED'
FROM (SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10 UNION ALL SELECT 11);

-- For USER SQLUSER002 (simulating activity then a significant decline).
-- Previous period: 20 calls.
INSERT INTO calls (user_id, call_date, started_at, duration, recording_status)
SELECT 'SQLUSER002',
       date('now', '-' || (abs(random()) % 29 + 31) || ' days'),
       datetime('now', '-' || (abs(random()) % 29 + 31) || ' days', '+' || (abs(random()) % 10) || ' hours'),
       abs(random())%50+10, 'PROCESSING'
FROM (SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10 UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20);

-- Current period: 4 calls (a significant decline from 20).
INSERT INTO calls (user_id, call_date, started_at, duration, recording_status)
SELECT 'SQLUSER002',
       date('now', '-' || (abs(random()) % 30) || ' days'),
       datetime('now', '-' || (abs(random()) % 30) || ' days', '+' || (abs(random()) % 10) || ' hours'),
       abs(random())%50+10, 'COMPLETED'
FROM (SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4);

-- Add a recent call for SQLUSER001 to test the integration_sync_failures query.
INSERT INTO calls (user_id, call_date, started_at, duration, recording_status) VALUES
('SQLUSER001', date('now', '-2 days'), datetime('now', '-2 days', '+15 hours'), 25, 'COMPLETED');
-- Add another recent call for SQLUSER002 that will be marked as a FAILED sync.
INSERT INTO calls (user_id, call_date, started_at, duration, recording_status) VALUES
('SQLUSER002', date('now', '-1 days'), datetime('now', '-1 days', '+10 hours'), 35, 'COMPLETED');


-- Create table for integration sync records.
CREATE TABLE integration_sync_records (
    sync_id INTEGER PRIMARY KEY AUTOINCREMENT,
    call_id INTEGER REFERENCES calls(call_id) UNIQUE, -- Each call has at most one sync record.
    sync_status TEXT CHECK (sync_status IN ('PENDING', 'SUCCESS', 'FAILED', NULL)), -- Allowed statuses.
    last_sync_attempt TEXT, -- Timestamp of the last sync attempt.
    error_message TEXT
);

-- Insert sample integration sync records.
-- Selecting existing call_ids to link sync records correctly.
-- Successful sync for an SQLUSER001 call.
INSERT INTO integration_sync_records (call_id, sync_status, last_sync_attempt, error_message)
SELECT call_id, 'SUCCESS', datetime(started_at, '+30 minutes'), NULL
FROM calls WHERE user_id='SQLUSER001' ORDER BY started_at LIMIT 1; -- Get the oldest call for this user.

-- Failed sync for a different SQLUSER001 call.
INSERT INTO integration_sync_records (call_id, sync_status, last_sync_attempt, error_message)
SELECT call_id, 'FAILED', datetime(started_at, '+5 minutes'), 'Error: CRM Connection Timeout.'
FROM calls WHERE user_id='SQLUSER001' ORDER BY started_at DESC LIMIT 1 OFFSET 1; -- Get a more recent call, skipping the latest.

-- Failed sync for the most recent SQLUSER002 call.
INSERT INTO integration_sync_records (call_id, sync_status, last_sync_attempt, error_message)
SELECT call_id, 'FAILED', datetime('now', '-1 hour'), 'Error: Invalid Record ID in target system.'
FROM calls WHERE user_id='SQLUSER002' ORDER BY started_at DESC LIMIT 1;
-- The recent call for SQLUSER001 (2 days ago) will intentionally not have a sync record to test the NULL case in the query.
"""
# Execute the DDL/DML script.
execute_sqlite_script(sqlite_setup_script_notebook, cursor_sql_notebook, conn_sql_notebook)
print("SQLite in-memory database for SQL examples is set up with tables and sample data.")

# Helper function to run an SQL query and display its results as a Pandas DataFrame.
def run_sql_query_notebook(query, connection):
    """
    Runs an SQL query against the given SQLite connection and returns results as a Pandas DataFrame.

    Args:
        query (str): The SQL query to execute.
        connection (sqlite3.Connection): The SQLite database connection.

    Returns:
        pd.DataFrame: A DataFrame containing the query results. Returns an empty DataFrame on error.
    """
    try:
        df = pd.read_sql_query(query, connection) # Use pandas to directly read SQL query results.
        return df
    except Exception as e:
        print(f"Error running SQL query: {e}")
        return pd.DataFrame() # Return an empty DataFrame if an error occurs.


SQLite script for database setup executed successfully.
SQLite in-memory database for SQL examples is set up with tables and sample data.


### SQL Example 1: API Performance Analysis (Adapted for SQLite)

This query is designed to identify slow-performing API calls. It analyses API request logs to calculate metrics such as average and maximum response times, the total number of calls, the count of error responses, and the error percentage for each API endpoint. The analysis is typically focused on a specific time range.

For this SQLite adaptation:
* The query structure remains similar to the PostgreSQL original.
* `SUM(CASE WHEN status_code >= 400 THEN 1 ELSE 0 END)` is used to count errors, which is a common and robust way in SQL.
* The date range for filtering (`BETWEEN '2024-07-01 00:00:00' AND '2024-07-07 23:59:59'`) uses the fixed dates from the sample data inserted in CELL 6 to ensure predictable results for demonstration.

The results are ordered by average response time in descending order to highlight the slowest endpoints first.



In [39]:
# CELL 7: Run API Performance Analysis Query

# This SQL query analyses the 'api_request_logs' table to find performance bottlenecks.
# It groups data by 'endpoint_path' and calculates key performance indicators.
query_api_perf_sqlite = """
SELECT
    endpoint_path,
    AVG(response_time_ms) as avg_response_time_ms, -- Average response time in milliseconds.
    MAX(response_time_ms) as max_response_time_ms, -- Maximum response time.
    COUNT(*) as call_count,                        -- Total number of calls to this endpoint.
    SUM(CASE WHEN status_code >= 400 THEN 1 ELSE 0 END) as error_count,  -- Count of error responses (status code >= 400).
    -- Calculate error percentage, handling division by zero by ensuring COUNT(*) > 0 (implicit in AVG).
    ROUND((SUM(CASE WHEN status_code >= 400 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) as error_percentage
FROM api_request_logs
WHERE
    -- Filter for a specific time range (using fixed dates from sample data for consistency).
    timestamp BETWEEN '2024-07-01 00:00:00' AND '2024-07-07 23:59:59'
    -- Optionally, uncomment to filter for a specific customer:
    -- AND customer_id = 'CUSTSQL01'
GROUP BY endpoint_path  -- Group results by API endpoint.
ORDER BY avg_response_time_ms DESC -- Order by average response time to prioritise slowest endpoints.
LIMIT 10;  -- Focus on the top 10 slowest endpoints.
"""

print("--- API Performance Analysis (SQLite) ---")
# Execute the query and store results in a DataFrame.
df_api_perf_sqlite = run_sql_query_notebook(query_api_perf_sqlite, conn_sql_notebook)

# Display the DataFrame if it's not empty.
if not df_api_perf_sqlite.empty:
    display(df_api_perf_sqlite)
else:
    print("No data returned for API Performance Analysis. Check sample data or query date range.")


--- API Performance Analysis (SQLite) ---


,endpoint_path,avg_response_time_ms,max_response_time_ms,call_count,error_count,error_percentage
0,/api/v1/orders,1200.0,1200,1,0,0.0
1,/api/v1/users,867.5,3000,4,1,25.0
2,/api/v1/products,625.0,750,2,1,50.0


### SQL Example 2: Declining Usage Detection (Adapted for SQLite)

This query aims to identify users whose engagement with a product or service (measured by call count in this example) has significantly declined. It compares user activity in two consecutive periods: the most recent 30 days versus the 30 days prior to that (i.e., 31-60 days ago). This proactive detection can help Customer Success teams intervene before a user churns.

Key aspects of the SQLite adaptation:
* **Common Table Expressions (CTEs):** `current_period` and `previous_period` are defined to calculate call counts for each user in the respective timeframes.
* **Date Functions:** SQLite's `date('now', '-X days')` is used to define these periods dynamically relative to the current date.
* **Percentage Change Calculation:** The percentage change in call count is calculated.
* **Filtering:** The query filters for users who were previously active (e.g., more than 10 calls in the previous period) and have experienced a significant drop in usage (e.g., at least a 30% decline).

The results are ordered by the percentage change, showing users with the most severe decline first.


In [40]:
# CELL 8: Run Declining Usage Detection Query

# This SQL query identifies users with declining activity by comparing call counts
# between the current 30-day period and the previous 30-day period.
query_declining_usage_sqlite = """
WITH current_period AS (
    -- Calculate call counts for each user in the last 30 days.
    SELECT
        user_id,
        COUNT(*) as call_count_current
    FROM calls
    WHERE call_date BETWEEN date('now', '-30 days') AND date('now')
    GROUP BY user_id
),
previous_period AS (
    -- Calculate call counts for each user in the period 31-60 days ago.
    SELECT
        user_id,
        COUNT(*) as call_count_previous
    FROM calls
    WHERE call_date BETWEEN date('now', '-60 days') AND date('now', '-31 days')
    GROUP BY user_id
)
-- Main query to join user information with period data and calculate usage change.
SELECT
    u.department,
    u.full_name,
    u.email,
    COALESCE(p.call_count_previous, 0) as previous_30d_calls, -- Handle users with no calls in previous period.
    COALESCE(c.call_count_current, 0) as current_30d_calls,   -- Handle users with no calls in current period.
    CASE
        WHEN p.call_count_previous > 0 THEN -- Avoid division by zero if no previous calls.
            ROUND((COALESCE(c.call_count_current, 0) - p.call_count_previous) * 100.0 / p.call_count_previous, 2)
        ELSE NULL -- Or set to 0 or a specific value if a new user's "change" is defined differently.
    END as percentage_change
FROM users u
LEFT JOIN current_period c ON u.user_id = c.user_id -- Join with current period activity.
LEFT JOIN previous_period p ON u.user_id = p.user_id -- Join with previous period activity.
WHERE (
    -- Filter for users who were significantly active previously (e.g., more than 10 calls).
    COALESCE(p.call_count_previous, 0) > 10 AND
    -- Filter for users whose current activity is less than 70% of their previous activity (i.e., >= 30% decline).
    (COALESCE(c.call_count_current, 0) < COALESCE(p.call_count_previous, 0) * 0.7)
)
ORDER BY percentage_change ASC; -- Show users with the most significant decline first.
"""
print("--- Declining Usage Detection (SQLite) ---")
# Execute the query.
df_declining_usage_sqlite = run_sql_query_notebook(query_declining_usage_sqlite, conn_sql_notebook)

# Display the results.
if not df_declining_usage_sqlite.empty:
    display(df_declining_usage_sqlite)
else:
    print("No users found with significant declining usage based on current data and criteria.")
    print("This could be due to sample data distribution or all active users maintaining usage.")

--- Declining Usage Detection (SQLite) ---


,department,full_name,email,previous_30d_calls,current_30d_calls,percentage_change
0,Support,SQL Bob,sql_bob@example.com,20,5,-75.0


### SQL Example 3: Integration Sync Failures (Adapted for SQLite)

This query is used to identify records (specifically, calls in this context) that have failed to synchronise with an external system or have not been synchronised recently. This is vital for troubleshooting data integration issues. The query focuses on calls made within the last 7 days.

SQLite adaptations:
* **Timestamp Comparison:** `datetime('now', '-7 days')` is used to define the 7-day window for recent calls.
* **Time Difference Calculation:** To calculate `minutes_since_last_attempt`, SQLite's `strftime('%s', ...)` function is used. This converts timestamp strings into Unix epoch seconds (number of seconds since 1970-01-01 UTC). The difference between `datetime('now', 'localtime')` (current local time as epoch seconds) and the `last_sync_attempt` (as epoch seconds) is then divided by 60 to get minutes. Using `'localtime'` for `'now'` helps align with typical server log timings if the database timestamps are also stored in local time.

The query retrieves call details along with sync status information, prioritising calls that are unsynced (`sync_status IS NULL`) or have a `FAILED` sync status.


In [41]:
# CELL 9: Run Integration Sync Failures Query

# This SQL query identifies calls from the last 7 days that have either failed to sync
# or have no sync record at all. It also calculates how long ago the last sync attempt was made.
query_sync_failures_sqlite = """
SELECT
    c.call_id,
    c.started_at,
    c.duration,
    c.recording_status,
    i.sync_status,
    i.last_sync_attempt,
    i.error_message,
    CASE
        WHEN i.last_sync_attempt IS NOT NULL THEN
            -- Calculate difference in minutes from 'now'.
            -- strftime('%s', ...) converts timestamp to Unix epoch seconds.
            -- Using datetime('now', 'localtime') to get current time adjusted for local timezone,
            -- assuming last_sync_attempt is also stored reflecting a similar timezone context.
            ROUND((strftime('%s', datetime('now', 'localtime')) - strftime('%s', i.last_sync_attempt)) / 60.0, 2)
        ELSE NULL -- No attempt means no minutes since.
    END as minutes_since_last_attempt
FROM calls c
LEFT JOIN integration_sync_records i ON c.call_id = i.call_id -- Join calls with their sync records.
WHERE
    c.started_at > datetime('now', '-7 days') -- Filter for calls started in the last 7 days.
    AND (i.sync_status IS NULL OR i.sync_status = 'FAILED') -- Select unsynced or failed syncs.
ORDER BY c.started_at DESC; -- Show the most recent problematic calls first.
"""

print("--- Integration Sync Failures (SQLite) ---")
# Execute the query.
df_sync_failures_sqlite = run_sql_query_notebook(query_sync_failures_sqlite, conn_sql_notebook)

# Display the results.
if not df_sync_failures_sqlite.empty:
    display(df_sync_failures_sqlite)
else:
    print("No integration sync failures found for calls in the last 7 days matching the criteria.")

--- Integration Sync Failures (SQLite) ---


,call_id,started_at,duration,recording_status,sync_status,last_sync_attempt,error_message,minutes_since_last_attempt
0,52,2025-05-14 20:12:36,35,COMPLETED,FAILED,2025-05-15 09:12:36,Error: Invalid Record ID in target system.,62.55
1,49,2025-05-14 12:12:36,12,COMPLETED,None,None,None,NaN
2,51,2025-05-14 01:12:36,25,COMPLETED,None,None,None,NaN
3,16,2025-05-12 17:12:36,15,COMPLETED,FAILED,2025-05-12 17:17:36,Error: CRM Connection Timeout.,3897.55
4,22,2025-05-10 13:12:36,34,COMPLETED,None,None,None,NaN


The final cell, CELL 10, closes the SQLite database connection that was used for the SQL examples. Since an in-memory database was used, this action effectively clears the database and its data from memory. If a file-based database had been used, closing the connection would simply release the file lock.

In [42]:
# CELL 10: Close the SQLite connection for SQL examples

# This cell closes the SQLite database connection that was opened for the SQL examples.
# It's good practice to explicitly close database connections when they are no longer needed,
# especially if it were a file-based database, to ensure all changes are written and resources are freed.
# For an in-memory database like the one used here (':memory:'), closing the connection
# effectively discards the database.

if conn_sql_notebook: # Check if the connection object exists.
    try:
        conn_sql_notebook.close() # Attempt to close the connection.
        print("\nSQLite in-memory connection for SQL examples closed successfully.")
    except Exception as e:
        # This might happen if the connection was already closed or is in an invalid state.
        print(f"\nError closing SQLite connection: {e}")
else:
    print("\nSQLite connection object for SQL examples does not exist or was already closed.")


SQLite in-memory connection for SQL examples closed successfully.


## Conclusion

This notebook has demonstrated several Python and SQL (adapted for SQLite) examples relevant to technical support and data analysis. By integrating these into a Jupyter/Colab notebook, we can combine explanations, code execution, and results in an interactive and shareable format.

**To use this notebook:**
* Run the cells sequentially from top to bottom.
* The Python examples create and use temporary files/databases (e.g., `sample_app_notebook.log`, `api_logs_notebook.db`) which are cleaned up at the end of their respective sections.
* The SQL examples use an in-memory SQLite database, meaning the data is not persisted if you close and reopen the notebook, but it ensures the notebook is self-contained and runnable anywhere without external database setup.
